This notebook is a Monte Carlo simulation of a LearnedLeague season. 

It ingests the following information:

1. The recent stats of all players in the league, defined as average TCA and DE in the three most recent seasons they have played in the past twelve months
2. The league's remaining schedule

Remaining TCA is projected based on a weighted average of the recent history and to-date performance that season, with an assumption towards mean regression that diminishes as the season continues.

Individual matchups are determined by projecting a sequence of integers 0-6 that sum to the remaining estimated TCA, running that number through a function assigning a random score weighted by your opponent's defensive efficiency, and compared to your opponent's score.

Final output is a frame sorted by median final placement, with additional promotion/relegation percentage chances added for public leagues.

*Model Limits*

1. Newer players are likely disadvantaged by this system, as it is to be expected that they will outperform their previous seasons as they learn to "read" the questions.
2. Players who have significantly improved themsleves between seasons will not be recognized as such until midseason or later. Additionally, because the model does not give special weight to the most recent season, recent improvement is not assumed to be permanent until repeated in multiple seasons.
3. Individual matchups are blind to the category stats of the individual players; it is my belief that a matchup between players with very different relative strengths is more likely to yield an upset than two players of relatively equivalent ability.
4. The "defensive table" and weighting curve used here are not based on meaningful research, and are merely my attempts to make reasonable assumptions about proper inputs.

Example output below is based on a run of Tundra A (my current rundle) as of MD0.

## User Inputs

In [2]:
league_type = 'public' #private #public

season = 100
league_name = 'Tundra'
rundle = 'A'
division = 0 #1, 2, 3 #zero for non-divided rundles

players = 28

#for public leagues; will be ignored in private leagues

promotion = 3
relegation = 4


### Package Installs + Setup

In [4]:
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import collections
from functools import reduce

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

import time
import random
from statistics import median

pd.set_option('display.max_columns', 30)

In [5]:
def formatted_lg(rundle, string, division):
    if (league_type == 'public') & (division == 0):
        return(rundle + '_' + string.capitalize())
    elif (league_type == 'public') & (division != 0):
        return(rundle + '_' + string.capitalize() + '_Div_' + str(division))
    elif league_type == 'private':
        return(string.title().replace(' ', '_'))
    
formatted_league = formatted_lg(rundle, league_name, division)

formatted_league

'A_Tundra'

In [6]:
def league_string(league_type, season, league_name, rundle, division):
    '''
    function that sets up the url for the league
    '''
    
    league_name = formatted_lg(rundle, league_name, division)
    
    if (league_type == 'public'):
        final_string = str(season) + '&' + league_name
        return(final_string)
        #if (division == 0):
        #    return(final_string)
        #else:
        #    return(final_string + '_Div_' + str(division))
    elif (league_type == 'private'):
        final_string = str(season) + '&' + league_name
        return(final_string)
    
def standings_url(string):
    return('https://learnedleague.com/standings.php?' + string)

In [7]:
league = league_string(league_type, season, league_name, rundle, division)
standings_url(league)

'https://learnedleague.com/standings.php?100&A_Tundra'

In [8]:
service = Service('chromedriver')
driver = webdriver.Chrome()
driver.get('https://learnedleague.com/')

In [9]:
credentials = pd.read_json('credentials.json')

login = driver.find_element(By.NAME, 'username')
login.send_keys(credentials['learnedleague']['username'])

pw = driver.find_element(By.NAME, 'password')
pw.send_keys(credentials['learnedleague']['password'])

clickable = driver.find_element(By.NAME, 'login')
clickable.click()

time.sleep(3)

driver.get(standings_url(league))

## Getting Baseline Info

In [10]:
def profile_getter(players):
    '''
    be sure to be navigated to the standings page already in selenium
    '''
    urls = []
    for i in range(1, players + 1):
        y = driver.find_element(By.XPATH, '//*[@id="lft"]/div[1]/table/tbody/tr[' + str(i) + ']/td[3]/a')
        url = y.get_attribute('href')
        urls.append(url)
        
    return(urls)

def schedule_generator():
    #run once at start of season to get schedule
    
    sched = pd.DataFrame(columns=['MD', 'P1', 'P2'])
    for k in range(1,26): #number of weeks in reg season less one
        time.sleep(1)
        if league_type == 'private':
            driver.get('https://learnedleague.com/schedule.php?' + str(season) + '&' + str(k) + '&' + formatted_league)
        elif league_type == 'public':
            driver.get('https://learnedleague.com/schedule.php?' + str(season) + '&' + str(k) + '&' + formatted_league)
            
        row_count = int((players / 2) + 1) #stops after the last row of players
        
        for i in range(1,row_count):
                x1 = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/table/tbody/tr[' + str(i) + ']/td[2]').text
                x2 = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/table/tbody/tr[' + str(i) + ']/td[4]').text
                newrow = {'MD': k, 'P1': x1, 'P2': x2}
                print(newrow)   
                sched = sched.append(newrow, ignore_index = True)
    
    sched.to_csv('LL' + str(season) + '_' + league_name + '_' + formatted_league + '_rundle_sched.csv')
    return sched

In [11]:
schedule_generator() # only need to run if you haven't before, can comment out otherwise

{'MD': 1, 'P1': 'BurgessD', 'P2': 'CheungE'}
{'MD': 1, 'P1': 'ChuHau', 'P2': 'Evaskis-GarrettC'}
{'MD': 1, 'P1': 'CraneN', 'P2': 'SmithC14'}
{'MD': 1, 'P1': 'EnsonR', 'P2': 'FiestaR'}
{'MD': 1, 'P1': 'FrithP', 'P2': 'BerkeA'}
{'MD': 1, 'P1': 'Gozlan Z', 'P2': 'GrubbsA'}
{'MD': 1, 'P1': 'HovelandR', 'P2': 'BonneauJ'}
{'MD': 1, 'P1': 'IngramB', 'P2': 'SteeleM'}
{'MD': 1, 'P1': 'LaBuaJ', 'P2': 'TemkinE'}
{'MD': 1, 'P1': 'PetersonDE', 'P2': 'JapingaM'}
{'MD': 1, 'P1': 'ReedN', 'P2': 'KligisJ'}
{'MD': 1, 'P1': 'RogersCP', 'P2': 'EgelmanS'}
{'MD': 1, 'P1': 'ZimmermanP', 'P2': 'GregoryB'}
{'MD': 1, 'P1': 'ZuffranieriJ', 'P2': 'RobinsonDrusoe'}
{'MD': 2, 'P1': 'BurgessD', 'P2': 'TemkinE'}
{'MD': 2, 'P1': 'ChuHau', 'P2': 'GregoryB'}
{'MD': 2, 'P1': 'CraneN', 'P2': 'EnsonR'}
{'MD': 2, 'P1': 'FrithP', 'P2': 'JapingaM'}
{'MD': 2, 'P1': 'Gozlan Z', 'P2': 'SteeleM'}
{'MD': 2, 'P1': 'HovelandR', 'P2': 'BerkeA'}
{'MD': 2, 'P1': 'IngramB', 'P2': 'BonneauJ'}
{'MD': 2, 'P1': 'LaBuaJ', 'P2': 'KligisJ'}
{'

{'MD': 13, 'P1': 'TemkinE', 'P2': 'CraneN'}
{'MD': 13, 'P1': 'ZuffranieriJ', 'P2': 'SmithC14'}
{'MD': 14, 'P1': 'BerkeA', 'P2': 'LaBuaJ'}
{'MD': 14, 'P1': 'BonneauJ', 'P2': 'BurgessD'}
{'MD': 14, 'P1': 'CheungE', 'P2': 'FrithP'}
{'MD': 14, 'P1': 'EgelmanS', 'P2': 'HovelandR'}
{'MD': 14, 'P1': 'Evaskis-GarrettC', 'P2': 'Gozlan Z'}
{'MD': 14, 'P1': 'GregoryB', 'P2': 'IngramB'}
{'MD': 14, 'P1': 'GrubbsA', 'P2': 'ZimmermanP'}
{'MD': 14, 'P1': 'JapingaM', 'P2': 'ReedN'}
{'MD': 14, 'P1': 'KligisJ', 'P2': 'CraneN'}
{'MD': 14, 'P1': 'RobinsonDrusoe', 'P2': 'ChuHau'}
{'MD': 14, 'P1': 'SmithC14', 'P2': 'EnsonR'}
{'MD': 14, 'P1': 'SteeleM', 'P2': 'RogersCP'}
{'MD': 14, 'P1': 'TemkinE', 'P2': 'PetersonDE'}
{'MD': 14, 'P1': 'ZuffranieriJ', 'P2': 'FiestaR'}
{'MD': 15, 'P1': 'BerkeA', 'P2': 'BurgessD'}
{'MD': 15, 'P1': 'BonneauJ', 'P2': 'RogersCP'}
{'MD': 15, 'P1': 'CheungE', 'P2': 'HovelandR'}
{'MD': 15, 'P1': 'EgelmanS', 'P2': 'IngramB'}
{'MD': 15, 'P1': 'Evaskis-GarrettC', 'P2': 'RobinsonDrusoe'}


,MD,P1,P2
0,1,BurgessD,CheungE
1,1,ChuHau,Evaskis-GarrettC
2,1,CraneN,SmithC14
3,1,EnsonR,FiestaR
4,1,FrithP,BerkeA
...,...,...,...
345,25,PetersonDE,SteeleM
346,25,ReedN,EgelmanS
347,25,RogersCP,ChuHau
348,25,SmithC14,TemkinE


In [12]:
driver.get(standings_url(league))
urls = profile_getter(players)

In [25]:
def last_five_seasons(number):
    minus_one = 'LL' + str(season - 1)
    minus_two = 'LL' + str(season - 2)
    minus_three = 'LL' + str(season - 3)
    minus_four = 'LL' + str(season - 4)
    minus_five = 'LL' + str(season - 5)
    return([minus_one, minus_two, minus_three, minus_four, minus_five])

def average_stats(urls):
    
    # pulls average TCA and DE for the most recent three seasons, with a tolerance of one missed season this year; 
    # you can mess with the "last four seasons" function above if you want to use a longer range of recent seasons
        
    stats = pd.DataFrame(columns=['Player', 'TCA', 'DE'])
    time.sleep(1)
    for url in urls:
        driver.get(url + "&2")
        time.sleep(1)
        
        try:
            name = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div[2]/h1').text
        except:
            name = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/div[2]/h1').text
        
        df = pd.read_html(driver.page_source)[4]
        df = df[df.Season.str.contains('LL')]
        df = df[df.Season.isin(last_five_seasons(season))]
        df = df[:3] #if they've played all four seasons, only take most recent three
        df['DE'] = [float(i) for i in df.DE]
        newrow = {'Player': name, 'TCA': round(df.TCA.mean()), 'DE': round(df.DE.mean(),3)}
        
        stats = stats.append(newrow, ignore_index = True)
        
    return(stats)

In [63]:
player_stats = average_stats(urls)

player_stats.sort_values('TCA', ascending=False)

,Player,TCA,DE
2,BurgessD,139,0.710
27,ZuffranieriJ,123,0.751
20,ReedN,123,0.745
5,CraneN,122,0.731
8,Evaskis-GarrettC,121,0.707
4,ChuHau,119,0.720
19,PetersonDE,118,0.624
9,FiestaR,116,0.674
3,CheungE,112,0.765
26,ZimmermanP,112,0.695


### Matchup Sim Functions

In [51]:
def defensive_table(de):
    '''
    Just a little something I cooked up to make defense kinda-sorta matter
    and convert correct answers into point totals
    '''
    probs_table = [{0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:0},
               {0: (2*.1667)*de, 1: (2*.3333)*de, 2: (2*.3333)*(1-de), 3: (2*.1667)*(1-de), 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:0},
               {0: 0, 1: (2*.1111)*de, 2: (2*.2222)*de, 3: .3333, 4: (2*.2222)*(1-de), 5: (2*.1111)*(1-de), 6: 0, 7: 0, 8: 0, 9:0},
               {0: 0, 1: 0, 2: (2*.08333)*de, 3: (2*.1667)*de, 4: (2*.25)*de, 5: (2*.25)*(1-de), 6: (2*.1667)*(1-de), 7: (2*.08333)*(1-de), 8: 0, 9:0},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: (2*.1111)*de, 5: (2*.2222)*de, 6: .3333, 7: (2*.2222)*(1-de), 8: (2*.1111)*(1-de), 9:0},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: (2*.1667)*de, 7: (2*.3333)*(de), 8: (2*.3333)*(1-de), 9:(2*.1667)*(1-de)},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:1}]
    return pd.DataFrame(probs_table)

def heads_up_points(score, DE):
    '''
    random points total for one shot
    '''
    
    probs = list(defensive_table(DE).loc[score])
    points = random.choices([0,1,2,3,4,5,6,7,8,9], weights=probs, k=1)
    
    return(points[0])

def heads_up_points_noisy(score, DE):
    '''
    random points total for one match with a little bit of bullshit
    because sometimes nonsense happens and we gotta account for that
    '''
    
    noise = random.randint(1,100)
    points = heads_up_points(score, DE)
    
    if (noise < 6) and (points > 0) and (score != 6):
        return(points - 1)
    if (noise > 95) and (points < 9) and (score != 0):
        return(points + 1)
    else:
        return(points)
    
def set_of_results(m, n):
    '''
    Generates a set of numbers of length (n) between 1 and 6, normally distributed,
    that sums up to total (m) 
    
    To be used to produce a random set of results for the rest of the season
    '''
    flag = 0
    while flag == 0:
        s = np.random.normal(m/n, 1, n) #mu = 1 is a number I made up as well that seemed to produce a coherent set of #s

        value_list = []

        for i in range(0,len(s)):
            j = round(s[i])
            if j > 6:
                j = 6
            if j < 0:
                j = 0
            value_list.append(j)

        if (sum(value_list) < (m + 1)) and (sum(value_list) > (m - 1)):
            flag = 1
        
    return(value_list)

def get_medians(df):
    '''sort that i like the most for this'''
    median_col = []
    
    for player in df.index:
        sim_pos = list(df.loc[player])
        all_results = []
        for i in range(1,len(sim_pos)+1):
            rel_list = list([i] * sim_pos[i-1])
            all_results = all_results + rel_list

        med = median(all_results)
        median_col.append(med)
        
    return(median_col)

### The Loop

The top commented out portion pulls the current standings and creates a stats that remain constant and unmodified in the following loops.

The i loop can be killed at any time to continue down the notebook. To continue appending after, run the "loop" cell again.

In [52]:
driver.get(standings_url(league))

sched_main = pd.read_csv('LL' + str(season) + '_' + league_name + '_' + formatted_league + '_rundle_sched.csv')
sched_main = sched_main[['MD', 'P1', 'P2']]

all_sims = pd.DataFrame(columns=['win', 'lose', 'tie', 'points', 'TCA', 'score', 'differential','place'])

#driver.get(standings_url(league))

to_date = pd.read_html(driver.page_source)[0][['Player', 'W', 'L', 'T', 'PTS', 'MPD', 'TMP', 'TCA', 'FL']]

## example of ad-hoc fix if player name is too long
to_date = to_date.replace({'Evaskis-Garre.': 'Evaskis-GarrettC'})

for col in ['W', 'L', 'T', 'PTS', 'MPD', 'TMP', 'TCA', 'FL']:
    to_date[col] = [float(i) for i in to_date[col]]

completed = int(to_date['W'][0] + to_date['L'][0] + to_date['T'][0])

if completed > 0:
    to_date['PER_DAY'] = round((to_date['TCA'] / (to_date['W'] + to_date['L'] + to_date['T'] - to_date['FL'])), 2)
    to_date['PER_DAY_PACE'] = to_date['PER_DAY'] * 25
    
else:
    to_date['PER_DAY'] = 0
    to_date['PER_DAY_PACE'] = 0

MD_weight = [0,	0,	0,	0,	0,	0,	0.03,	0.07,	0.11,	0.17,\
             0.24,	0.32,	0.4,	0.5,	0.6,	0.66,	0.76,\
             0.83,	0.89,	0.93,	0.97,	1,	1,	1,	1,	1]


In [108]:
#begin generating sims

session_sims = pd.DataFrame(columns=['win', 'lose', 'tie', 'points', 'TCA', 'score', 'differential','place'])

for i in range(0, 10000): #you can lower this if you want, or pause this cell and continue running after
    seed = random.randint(0,5000000000) #move to higher level when processing

    sim = player_stats
    sim = sim.merge(to_date, on='Player', how='left', suffixes=['','_todate'])
    
    sim['weighted_TCA'] = (sim['TCA'] * (1- MD_weight[completed])) + (sim['PER_DAY_PACE'] * (MD_weight[completed]))
    sim['weighted_TCA'] = [round(i, 0) for i in sim.weighted_TCA]        
    sim['TCA_reversion'] = [round(i - (j*completed),0) for i, j in zip(sim.weighted_TCA, sim.PER_DAY)] #round(i - (j*completed),0)
    sim['random_matches'] = [set_of_results(i, 25 - completed) for i in sim.TCA_reversion]

    sched = sched_main

    sched = sched_main[sched_main['MD'] > completed]
    sched = sched.reset_index()

    sched['P1_TCA'] = [sim[sim.Player == i]['random_matches'].iloc[0][j-completed-1] for i, j in zip(sched.P1, sched.MD)]
    sched['P2_TCA'] = [sim[sim.Player == i]['random_matches'].iloc[0][j-completed-1] for i, j in zip(sched.P2, sched.MD)]

    sched['P1_DE'] = [player_stats[player_stats.Player == i]['DE'].iloc[0] for i in sched.P1]
    sched['P2_DE'] = [player_stats[player_stats.Player == i]['DE'].iloc[0] for i in sched.P2]

    sched['P1_score'] = [heads_up_points_noisy(i,j) for i,j in zip(sched.P1_TCA, sched.P2_DE)]
    sched['P2_score'] = [heads_up_points_noisy(i,j) for i,j in zip(sched.P2_TCA, sched.P1_DE)]

    P1_win = []
    P2_win = []

    P1_tie = []
    P2_tie = []

    P1_lose = []
    P2_lose = []

    for i in range(0, len(sched)):
        if sched.P1_score[i] > sched.P2_score[i]:
            P1_win.append(1)
            P2_win.append(0)
            P1_tie.append(0)
            P2_tie.append(0)
            P1_lose.append(0)
            P2_lose.append(1)
        elif sched.P1_score[i] < sched.P2_score[i]:
            P1_win.append(0)
            P2_win.append(1)
            P1_tie.append(0)
            P2_tie.append(0)
            P1_lose.append(1)
            P2_lose.append(0)
        else:
            P1_win.append(0)
            P2_win.append(0)
            P1_tie.append(1)
            P2_tie.append(1)
            P1_lose.append(0)
            P2_lose.append(0)

    sched['P1_win'] = P1_win
    sched['P2_win'] = P2_win

    sched['P1_lose'] = P1_lose
    sched['P2_lose'] = P2_lose

    sched['P1_tie'] = P1_tie
    sched['P2_tie'] = P2_tie

    sched['differential'] = sched.P1_score - sched.P2_score

    long = pd.DataFrame(columns=['MD', 'Player', 'TCA', 'score', 'win', 'lose', 'tie', 'differential'])

    for i in range(0, len(sched)):
        newrow_1 = {'MD': sched.MD[i], 'Player': sched.P1[i], 'TCA': sched.P1_TCA[i],\
                    'score': sched.P1_score[i], 'win': sched.P1_win[i], 'lose': sched.P1_lose[i],\
                    'tie': sched.P1_tie[i], 'differential': sched.differential[i]}
        newrow_2 = {'MD': sched.MD[i], 'Player': sched.P2[i], 'TCA': sched.P2_TCA[i],\
                    'score': sched.P2_score[i], 'win': sched.P2_win[i], 'lose': sched.P2_lose[i],\
                    'tie': sched.P2_tie[i], 'differential': (sched.differential[i] * -1)}
        long = long.append(newrow_1, ignore_index = True)
        long = long.append(newrow_2, ignore_index = True)


    simmed_standings = long.groupby(by="Player").sum()

    simmed_standings['points'] = (simmed_standings['win'] * 2) + simmed_standings['tie']
    simmed_standings = simmed_standings[['win', 'lose', 'tie', 'points', 'TCA', 'score', 'differential']]

    simmed_standings = simmed_standings.sort_values(['points', 'differential', 'score', 'TCA'], ascending=False)
    simmed_standings['place'] = range(1,len(simmed_standings)+1)

    simmed_standings.merge(to_date, on='Player', how='left', suffixes=['','_todate'])

    spt = simmed_standings.merge(to_date, on='Player', how='left', suffixes=['','_todate']) #simmed plus todate

    simmed_latest = pd.DataFrame(columns=['Player', 'W', 'L', 'T', 'Pts', 'TCA', 'TMP', 'Diff', 'FL'])

    #simmed_standings['place'] = range(1,len(simmed_standings)+1)

    simmed_latest['Player'] = spt['Player']
    simmed_latest['W'] = spt['win'] + spt['W']
    simmed_latest['L'] = spt['lose'] + spt['L']
    simmed_latest['T'] = spt['tie'] + spt['T']
    simmed_latest['Pts'] = spt['points'] + spt['PTS']
    simmed_latest['TCA'] = spt['TCA'] + spt['TCA_todate']
    simmed_latest['TMP'] = spt['score'] + spt['TMP']
    simmed_latest['Diff'] = spt['differential'] + spt['MPD']
    simmed_latest['FL'] = spt['FL']

    simmed_latest = simmed_latest.sort_values(['Pts', 'Diff', 'TMP', 'TCA'], ascending=False)
    simmed_latest['place'] = range(1,len(simmed_latest)+1)

    simmed_latest = simmed_latest.set_index(['Player'])

    print(simmed_latest[['place']])

    session_sims = pd.concat([session_sims, simmed_latest])

                  place
Player                 
BurgessD              1
Evaskis-GarrettC      2
FiestaR               3
PetersonDE            4
CraneN                5
Gozlan Z              6
RogersCP              7
ChuHau                8
KligisJ               9
TemkinE              10
EgelmanS             11
ZuffranieriJ         12
CheungE              13
HovelandR            14
ZimmermanP           15
BerkeA               16
SmithC14             17
BonneauJ             18
IngramB              19
ReedN                20
GregoryB             21
EnsonR               22
SteeleM              23
RobinsonDrusoe       24
GrubbsA              25
FrithP               26
LaBuaJ               27
JapingaM             28


KeyboardInterrupt: 

In [107]:
all_sims = pd.concat([all_sims, session_sims])
sim_count = len(all_sims) / players #number of iterations run
sim_count

424.0

In [99]:
all_sims['player'] = all_sims.index
wide = pd.pivot_table(all_sims, values = ['place'], index = ['player'], columns=all_sims['place'].values, aggfunc='count', fill_value=0)

In [100]:
if league_type == 'public':
    promotion_frame = wide['place'][range(1,promotion+1)]
    promotion_frame['promoted'] = [promotion_frame.iloc[i].sum(axis=0) for i in range(0, len(promotion_frame))]
    relegation_frame = wide['place'][range(players-relegation + 1, players + 1)]
    relegation_frame['relegated'] = [relegation_frame.iloc[i].sum(axis=0) for i in range(0, len(relegation_frame))]
    wide['promoted'] = promotion_frame['promoted']
    wide['relegated'] = relegation_frame['relegated']

In [101]:
wide['median_pos'] = get_medians(wide)

In [104]:
wide = wide.sort_values('median_pos', ascending=True)

wide[['median_pos', 'place']]

median_pos place                                              \
                                1   2   3   4   5   6   7   8   9  10  11  12   
player                                                                          
BurgessD                5.0   312  31   5   1   2   1   0   0   0   0   0   0   
ReedN                   6.0    11  70  60  54  34  25  22  15  11  10  11   6   
ZuffranieriJ            7.0     8  74  62  44  37  22  23  19  15   8  14   7   
CraneN                  7.0     7  53  46  49  40  33  22  27  19  13  10   7   
Evaskis-GarrettC        7.0     7  40  48  40  35  35  25  24  22  24  13  10   
ChuHau                  8.0     4  30  35  28  28  51  32  19  21  16  22  15   
PetersonDE             10.0     2   7  13  22  19  30  25  29  31  24  25  11   
CheungE                12.0     0  10   7  12  21  15  19  27  21  25  15  20   
FiestaR                12.0     0   8  14  16  17  17  26  16  23  19  18  21   
ZimmermanP             13.0     0   7  11  13  15  11  20  24  25  20  19  15   
GregoryB               13.0     0   3   6  13  15  15  18  11  20  24  18  29   
RogersCP               15.0     0   4  11   6  16  16  10  13  19  16  13  15   
TemkinE                15.5     0   2   4  10  12   9  11  17  14  25  21  15   
BerkeA                 16.0     0   3   4   5   7  16  12  17  15  16  12  21   
EgelmanS               16.0     0   2   6   7  12   9   9  10  13   8  19  25   
BonneauJ               16.0     1   1   5   5   8   6  12  16  13  18  22  17   
GrubbsA                16.0     0   1   2   3   9  12  15  13  14  13  16  18   
SmithC14               17.0     0   1   5   6   3   9  18  13  11  13  11  17   
HovelandR              17.5     0   0   1   5   7   8  10  12  13   8  16  18   
IngramB                19.0     0   2   1   4   6   3   9  15   7  12  11  14   
Gozlan Z               21.0     0   2   3   3   5   2   2   6  10   7  11  12   
LaBuaJ                 22.0     0   0   2   2   1   5   3   3   4  10  11   8   
EnsonR                 24.0     0   1   0   2   1   0   5   1   0   5   2   9   
KligisJ                25.0     0   0   1   0   0   0   0   3   5   7   7   9   
FrithP                 26.0     0   0   0   1   0   2   2   1   3   2   3   4   
RobinsonDrusoe         26.0     0   0   0   1   1   0   2   0   2   3  10   3   
JapingaM               27.0     0   0   0   0   0   0   0   1   0   3   2   5   
SteeleM                28.0     0   0   0   0   1   0   0   0   1   3   0   1   

                                                                              \
                  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27   
player                                                                         
BurgessD           0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
ReedN              7   5   3   3   5   0   0   0   0   0   0   0   0   0   0   
ZuffranieriJ       7   4   2   4   1   0   0   0   0   0   0   1   0   0   0   
CraneN             8   1   4   0   5   4   2   1   0   1   0   0   0   0   0   
Evaskis-GarrettC   3   4   9   4   3   2   2   0   1   0   0   1   0   0   0   
ChuHau            12   5   7   6   8   3   5   0   2   1   0   1   0   1   0   
PetersonDE        18  17  16  10   5  11   5   8   6   5   4   3   3   1   2   
CheungE           19  21  17  16  17  13  14  11   9  10   4   3   4   1   1   
FiestaR           24  22  20  10   8  10  18  10  13   8   4   3   4   3   0   
ZimmermanP        25  18  14  15  15  16   8  16   8  12   6  11   3   3   2   
GregoryB          18  22  13  23  16  17  14  16   9   8   4   5   8   4   1   
RogersCP          14  14  29  14  19  23  15  16  15  14  16   7   9   2   2   
TemkinE           14  22  16  16  20  13  11  17  17  17  15   8  10   6   4   
BerkeA            21  19  19  12  15  22  18  22  10  15  13  14  11   4   7   
EgelmanS          15  20  22  25  13  13  20  17  15  15  15   9  12  13   5   
BonneauJ          16  17  28  15  21   9  22  13  24   8  15  12 

In [105]:
wide = wide.apply(lambda x: x/sim_count)

if league_type == 'public':
    wide[['promoted', 'relegated', 'place']].to_csv('probs_LL' + league + '_md' + str(completed) + '.csv')
elif league_type == 'private':
    wide[['promoted']].to_csv('probs_LL' + str(season) +'_' + league + '_md' + str(completed) + '.csv')